# Vinyl For Beginners

This is a slow, step-by-step tutorial for the record library Vinyl.
Records are ubiquituous in programming.
Vinyl opens up a window to type level programming with records in Haskell.
This means that _at compile time_, record types can be augmented with new fields, subsets can be taken, we can check if a given field is in the record and so on.
Runtime access to non-existing fields, and other sources of error can thus be avoided from the beginning on, and the compiler gives instant feedback in the IDE.

## What are Anonymous Extensible Records and why do we need them?

Wikipedia introduces records like this:

> In computer science, a record (also called a structure, struct, or compound data) is a basic data structure. Records in a database or spreadsheet are usually called "rows".
> A record is a collection of fields, possibly of different data types, typically in fixed number and sequence.

In Haskell, the standard way to define a record type looks like this:

In [1]:
data MyRecord = MyRecord {
        name :: String,
        age :: Int
  }

This binds the name `MyRecord` to a new type that is a record with two fields `name` of type `String` and `age` of type `Int`.
We can construct a value of this type:

In [2]:
myrecord = MyRecord {age = 1, name="me"}

Without the field names, standard haskell records are similar to a tuple, a datastructure that could be any combination of `String` and `Int`.
However, besides the field names, there is one big conceptual difference between `MyRecord` and a tuple `(String, Int)`:
`MyRecord` is a _named_ type whereas `(String, Int)` is _anonymous_, it doesn't have a name that refers to it but is just defined on the fly in terms of its types.
For example, we can write:

In [3]:
foo :: (String, Int) -> (String, Int, Int)

: 

This is what allows to use tuples in a flexible manner.
Wouldn't it be nice to define records on the fly as well?
Then there is another point of operations.
What if I wanted to write a function that takes field names and returns a subset of a record as a new record type with the corresponding fields?
How about a function that appends a new field to a record?
Instead of having statically defined data types like `MyRecord`, we thus need more flexible types with which we can compute on the fly.
And this is where Vinyl comes into play.

## First Steps with Type Level Records

The previous section has made it clear that we will need to do some type level programming.
We activate two essential Haskell language extensions that give us basic capacities to do so:

In [4]:
:ext DataKinds
:ext TypeOperators

`DataKinds` allows us to define new types.
`TypeOperators` gives us the possibility to compute with these types using operators.
Now that we are ready, let's get started and import the Vinyl Record type `Rec`, together with its two constructors `RNil` that builds an empty record and `:&` that prepends a value to another record.

In [5]:
import Data.Vinyl (Rec(RNil, (:&)))

Think about a record as a list, a type level list that contains the types of the various fields as its elements and other things such as field names.
But before starting, how about having a look at what we just imported.
Here is the _kind signature_ of `Rec`:

In [6]:
:k Rec

Rec :: forall u. (u -> *) -> [u] -> *

`Rec` is a type constructor, that means it builds new types on the fly, similar to the tuple operator `(,)`, and this is what we want.
Note that this is _not_ a named type as `MyRecord`.
But the kind signature of `Rec` looks weird.
After what we have discussed, a record is basically a type level collection of fields, but the first argument of `Rec` is a type level _function_ `u -> *`.
This means that it is a function from types of kind `u`, that is the types that are stored in the type level list, to types of kind `*` which just means any concrete type like `Int`, `String`, `MyRecord` and so on.
This type level function is a powerful particularity of Vinyl.
It is called the _Interpretation Functor_ and it allows to attach effects, or other values to the types that are stored in the record.
For example, we could wrap all record values in a `Maybe` which means that they could also be `Nothing` in addition to the values of their type.
To get started we chose the simplest type level function imaginable, the identiy, and import it.
Be careful to use the one provided by Vinyl because it has some type classes attached to it that the equivalent one from `Data.Functor.Identity` doesn't have:

In [7]:
import Data.Vinyl.Functor (Identity(Identity))

Let's see how to construct an anonymous extensible record type:

In [8]:
example1 :: Rec Identity [String, Int]
example1 = Identity "Bob" :& Identity 22 :& RNil

In [9]:
example1

{"Bob", 22}

The type level list corresponds to `MyRecord` although it doesn't have field names yet.
But we can already append new fields to it:

In [10]:
Identity 15 :& example1

{15, "Bob", 22}

What is amazing is that this is not simply a list with any number of elements.
What we have got now is this type:

In [11]:
:t Identity 15 :& example1

Identity 15 :& example1 :: forall r. Num r => Rec Identity '[r, String, Int]

But we can do more things.
For example, we can select a subset of the original record _simply by specifying the type_ with the function rcast:

In [12]:
import Data.Vinyl (rcast)
rcast example1 :: Rec Identity '[Int]
rcast example1 :: Rec Identity '[String]

{22}

{"Bob"}

If we try to access a member of the collection that doesn't exist, we get a compile time error:

In [13]:
rcast example1 :: Rec Identity '[Char]

: 

Afterall that's where type checking happens.
But we are now slowly reaching the limits of a simple collection of fields.
What happens if two fields have the same type?
Defining such a type works well:

In [14]:
example2 :: Rec Identity [String, String]
example2 = Identity "Mister" :& Identity "Bob" :& RNil

In [15]:
example2

{"Mister", "Bob"}

But selecting subsets reaches its limits:

In [16]:
rcast example1 :: Rec Identity '[String]

{"Bob"}

This is what we need _named_ fields for, and that requires type level strings called "Symbols" in Haskell.

## Playing with the Interpretation Functor

Thus far we have left the interpretation functor a simple `Identity`.
This meant that the types that are stored in the type level list, are also interpreted as what they are.
But what can we do if we change this?

In [17]:
example3 :: Rec Maybe [String, Int]
example3 = Just "Bob" :& Nothing :& RNil

example3

{Just "Bob", Nothing}

Recall the kind signature of `Rec` and the type signature of the constructor `:&` and `RNil`:

In [18]:
:k Rec

Rec :: forall u. (u -> *) -> [u] -> *

What corresponds to what in our example above?
First we have `Maybe ~ (u -> *)` that maps types of kind `u` to concrete types.
Then we have `[String, Int] ~ [u]`, and because `String` and `Int` are just of kind `*` we see that `u ~ *`.
So we really have a Functor `Maybe :: * -> *` here that wraps our concrete types into another, enriched concrete type.
In this case, the enrichment just means that we have access to an extra value `Nothing` besides the values of the type itself.
Let's look at the constructor now:

In [19]:
:t RNil
:t (:&)

RNil :: forall (a :: * -> *). Rec a '[]

(:&) :: forall (a :: * -> *) r (rs :: [*]). a r -> Rec a rs -> Rec a (r : rs)

`RNil` is simple to understand.
It stands for an empty record with arbitrary Functor `a` that can be determined via type inference.
`(:&)`, the cons operator, is only a bit more involved.
It takes `a r`, a value `r` wrapped in a functor `a`, and attaches it to an existing `Rec` with the same functor `a`.
That's why we need to wrap even pure values into an `Identity` functor on construction.
We are now ready to understand named fields that are probably most important use case.

## Named Fields

Key to understanding records with named fields is the Functor `ElField` that we import like this, together with the operator `:::` whose meaning will become clear in the following:

In [20]:
import Data.Vinyl ((:::), ElField(Field))

And here is how these are used:

In [21]:
example2 :: Rec ElField ["name" ::: String, "age" ::: Int]
example2 = Field "Bob" :& Field 22 :& RNil

In [22]:
example2

{name :-> "Bob", age :-> 22}

To understand how this works, let's look at the kind signature of `:::` first:

In [23]:
:k (:::)

(:::) :: forall k k1. k -> k1 -> (k, k1)

This type operator takes a type of kind `k` and a type of kind `k1` and puts them together in a type level tuple, that is a new type with its own kind.
This tuple is defined by a datatype and a type level string aka Symbol.
In our case this means that we build `("Name", String)` and `("Age", Int)` as type level tuples that live in our record.
We now make use of the interpretation functor, in this case `Elfield`.
Its kind signature looks like this:

In [24]:
:k ElField

ElField :: (Symbol, *) -> *

Remember, it is a type level function with kind signature `u -> *` where `u` corresponds to the kind that is stored in the record, a type level tuple with a symbol as its first and a concrete type as its second element.
Elfield wraps these exotic types and returns another enriched concrete type, similar to Maybe that wraps a concrete type and returns an enriched concrete type.
Elfields can be constructed with the unique `Field` constructor:

In [25]:
:t Field

Field :: forall (s :: Symbol) t. KnownSymbol s => t -> ElField '(s, t)

It takes a value and attaches it with a type level symbol.
In our construction above, this type level symbol is automatically determined via type inference.
Let's try to specify it on our own:

In [26]:
example4 = (Field "Bob" :: ElField '("name", String)) :& (Field 22 :: ElField '("age", Int)) :& RNil
example4
:t example4

{name :-> "Bob", age :-> 22}

example4 :: Rec ElField '[ '("name", String), '("age", Int)]

Cool, this works!
Let's see if we can still select subrecords:

In [27]:
rcast example2 :: Rec ElField '["name" ::: String]

{name :-> "Bob"}

In [28]:
:ext FlexibleContexts
import Data.Vinyl (rput, (=:))
example3 = rput (Field ("Alice") :: ElField ("name" ::: String)) example2

In [29]:
example3

{name :-> "Alice", age :-> 22}

In [30]:
:ext OverloadedLabels
:ext TypeFamilies
import Data.Vinyl (rputf)
example4 = rputf #name "Martha" example2
example4

{name :-> "Martha", age :-> 22}

## Construction Frenzy

In [31]:
import Data.Vinyl (FieldRec)

t :: FieldRec ["name" ::: String, "age" ::: Int]
t = Field "mat" :& Field 32 :& RNil
t

{name :-> "mat", age :-> 32}

In [32]:
import Data.Vinyl (xrec)

t :: FieldRec ["name" ::: String, "age" ::: Int]
t = xrec ("mat", 32)
t

{name :-> "mat", age :-> 32}

In [33]:
import Data.Vinyl.FromTuple (namedArgs)

t :: FieldRec ["name" ::: String, "age" ::: Int]
t = namedArgs (#age =: (23 :: Int), #name =: "Joe")
t

{name :-> "Joe", age :-> 23}

In [34]:
:ext OverloadedStrings

import Data.Text (Text)
import Data.Aeson (object, FromJSON(parseJSON), withObject, (.:), Value, (.=))
import Data.Aeson.Types (Parser, parseEither)

t :: FieldRec ["name" ::: String, "age" ::: Int]
t = case fromObject $ object [ "age" .= (23 :: Int), "name" .= ("Joe" :: Text) ] of
        Right x -> x
        Left _ -> error "couldn't convert record"
    where
      fromObject = parseEither $ withObject "Record" $ \o -> do
         age <- o .: "age"
         name <- o .: "name"
         return $ namedArgs (#age =: (age :: Int), #name =: (name :: String))

t

{name :-> "Joe", age :-> 23}

## Deconstruction Frenzy (?)

How about deconstructing a record?
Here is the straight forward approach:

In [35]:
trans1 :: Rec Identity '[Int, String] -> Rec Identity '[String, String]
trans1 (Identity a :& Identity b :& RNil) = Identity (show a) :& Identity (show b) :& RNil

trans1 $ xrec (23, "joe")

{"23", "\"joe\""}

## Functions over records

You'd hope that mapping over a record is as simple as using `<$>` but unfortunately it isn't.
This is largely due to the fact that we are dealing with a _heterogeneous_ collection of elements here.
The function that is applied over the record thus changes its type depending on the element it is applied to.

In [36]:
import Data.Vinyl.Class.Method (mapFields)
import Data.Vinyl (rmap)

t = xrec (1.0, 1) :: FieldRec ["a" ::: Double, "b" ::: Int]
t

{a :-> 1.0, b :-> 1}

## Mapping over Records (or Playing with the Interpretation Functor 2)

Since this is slightly more involved, we'll go through two examples.
Let's start with serializing records to JSON.
Say we know how to serialize each individual `ElField` in the record.
This can, for example be implemented with this type class:

In [37]:
import Data.Aeson (ToJSON(toJSON))

instance ToJSON a => ToJSON (Identity a) where
  toJSON (Identity x) = toJSON x

Now how do we cannot simply map this function over the fields

In [38]:
t = xrec ("Joe", 23) :: Rec Identity '[String, Int]
toJSON <$> t

: 

The compiler tells us `Couldn't match kind ‘[*]’ with ‘*’`.
And this makes sense because the kind `[*]` is just something different than the kind `*`.
In other words, our mapping attempt tried to apply `toJSON` to a type of kind `[*]` but that is not how it is defined.
It is defined as an operation on the individual fields wrapped in the `Identity` functor.
But how do we do it then?

One approach is to change the interpretation functor.
Vinyl provides a method `rmapMethod1` for this:

In [39]:
import Data.Vinyl.Class.Method (RecMapMethod1(rmapMethod1))

:t rmap

rmap :: forall (rs :: [*]) (f :: * -> *) (g :: * -> *). RMap rs => (forall x. f x -> g x) -> Rec f rs -> Rec g rs

We can see that it takes a function that transforms an interpretation function `f` into another one `g`, for any value of `x`.
How would this look like for `Identity`?
Let's try to wrap all values into a `Maybe` functor.
For example, setting them to `Just` and then their value.
This function is easy to write:

In [40]:
mapper :: Identity x -> Maybe x
mapper (Identity x) = Just x

Can we map this over a record?

In [41]:
rmap mapper t

{Just "Joe", Just 23}

Excellent, this works!
But isn't there a fundamental difference between wrapping a type and _changing the type of something_?
Indeed, the type signature of `rmap` requires the types `rs` of the record fields to remain unchanged.
Here is a trick around this, the `Const` functor:

In [42]:
import Data.Vinyl.Functor (Const(Const), getConst)

mapper2 :: Identity a -> Const String a
mapper2 (Identity x) = Const "hey"

rmap mapper2 t

{(Const "hey"), (Const "hey")}

Wow wait.
Isn't this cheating?
Let's look at the type of this new record:

In [43]:
:t (rmap mapper2 t)

(rmap mapper2 t) :: Rec (Const String) '[String, Int]

We see that, although all elements appear to be strings, the original type of the field _hasn't changed_.
However their interpretation _has changed_.
We can get the final values like this:

In [44]:
import Data.Vinyl (rget)

getConst (rget (rmap mapper2 t) :: Const String String)
getConst (rget (rmap mapper2 t) :: Const String Int)

"hey"

"hey"

Instead of having a `Maybe` value attached, we now have a constant value attached!
Can we do this also with a type class?

In [45]:
mapper3 :: Show a => Identity a -> Const String a
mapper3 (Identity x) = Const (show x)

This works fine.
However, this one doesn't:

In [46]:
rmap mapper3 t

: 

We need more magic to make this work.
We need a mapping function that can ensure the correct class constraints.
And Vinyl provides such a method:

In [47]:
:ext TypeApplications

import Data.Vinyl (rmapMethod)

rmapMethod @Show mapper3 t

{(Const "\"Joe\""), (Const "23")}

The class that is to be constraint is fed to `rmapMethod` via Typeapplications.
Changing the interpretation functor might seem a bit ugly.
Another way of mapping exists:

In [48]:
:t (rmapMethod @Show mapper3)

(rmapMethod @Show mapper3) :: forall (ts :: [*]). RecMapMethod Show Identity ts => Rec Identity ts -> Rec (Const String) ts

In [49]:
t :: FieldRec ["name" ::: String, "age" ::: Int]
t = xrec ("joe", 23)
t

{name :-> "joe", age :-> 23}

In [50]:
rmapMethod @Show mapper3 t

: 

Well this is annoying.
Let's see what we can do about it:

In [68]:
transform :: (RecMapMethod1 Show ElField rs) => Rec ElField rs -> Rec (Const String) rs
transform = rmapMethod1 @Show (Const . show)

transform t

{(Const "name :-> \"joe\""), (Const "age :-> 23")}

Well this is not quit what we want.
It transformed the whole `ElField` to a String.
How can we make this better?

In [67]:
import Data.Vinyl (getField)

transform :: (RecMapMethod1 Show ElField rs) => Rec ElField rs -> Rec (Const String) rs
transform = rmapMethod1 @Show (Const . show . getField)

transform t

: 

S... the types got us again.
The issue is that `RecMapMethod1 c f` expects the _constraint `c` to apply to `f a` for each `a`_, as `Show` does because `ElField (s,a)` has a show instance.
In this case, however, with `x :: ElField (s,a)` we apply show to `getField x`, not to `x` itself.
Let's setup a different instance:

In [58]:
:ext FlexibleInstances

class MyShow a where
    myshow :: a -> String

instance Show a => MyShow (ElField '(s,a)) where
    myshow x = show $ getField x

:t myshow (Field "hey" :: ElField ("hey" ::: String))

myshow (Field "hey" :: ElField ("hey" ::: String)) :: String

In [59]:
mapper5 :: (RecMapMethod1 MyShow ElField a) => Rec ElField a -> Rec (Const String) a
mapper5 = rmapMethod1 @MyShow (Const . myshow)

In [70]:
b = mapper5 t
b

{(Const "\"joe\""), (Const "23")}

What is the easiest way to get these values out?

In [111]:
import Data.Vinyl (rfoldMap)

rfoldMap (\x -> [getConst x]) b

["\"joe\"","23"]

In [112]:
:t (rfoldMap (\x -> [getConst x]))

(rfoldMap (\x -> [getConst x])) :: forall (rs :: [*]) a. RFoldMap rs => Rec (Const a) rs -> [a]